In [1]:
opt = {'switch':20000}
opt.update({'lr':1e-4})

opt['dir_root']='/home/mohit/Music/attention_low_light/'
opt['exp_name'] = 'AVG-on-1-10-sec-raw2rgb-patch512-fullVGG-RELU_remove' # the weights can be found in the 'ablations/weights' folder

opt['gpu'] = "1"
opt['epochs'] = 1000000
opt['batch_size'] = 1
opt['Shuffle'] = False
opt['Pin_memory'] = True
opt['workers'] = 1
opt['patch'] = 512

opt['fig_freq'] = 2000
opt['save_freq'] = [2,200000,400000,450000,500000]
opt['text_prnt_freq']=2000

opt['fig_size'] = 5

In [2]:
import random
import pandas as pd
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torchvision.transforms.functional as Ft

import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import skimage
import time
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import normalized_root_mse as NRMSE
from torch.autograd import Variable
from math import exp
import math
import rawpy
import glob
import imageio


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
##os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["CUDA_VISIBLE_DEVICES"]=opt['gpu']

In [3]:
num = random.randint(0, 20)
num = (num//3)*3 
print(num)

0


In [4]:
class get_data(Dataset):
    """Loads the Data."""
    
    def __init__(self,opt):
        self.train_files = glob.glob('/media/mohit/data/mohit/chen_dark_cvpr_18_dataset/Sony/short/1*_00_0.1s.ARW')
#         self.train_files = self.train_files + glob.glob('/media/mohit/data/mohit/chen_dark_cvpr_18_dataset/Sony/short/2*_00_0.1s.ARW')


        self.gt_files = []
        for x in self.train_files:
            self.gt_files =self.gt_files+ glob.glob('/media/mohit/data/mohit/chen_dark_cvpr_18_dataset/Sony/long/*'+x[-17:-12]+'*.ARW')
        
        self.to_tensor = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        
        self.opt = opt
        
    def __len__(self):
        return len(self.gt_files)

    def __getitem__(self, idx):
        
       
        raw = rawpy.imread(self.gt_files[idx])
        
        img_gt = raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16).copy()
        img_gtt=np.float32(img_gt/65535.0)

        raw.close()
        
        raw = rawpy.imread(self.train_files[idx])
        img = raw.raw_image_visible.astype(np.float32).copy()
        raw.close()
        
        img_loww = (np.maximum(img - 512,0)/ (16383 - 512))
        H,W = img_loww.shape
        
        ##############################################################################
        
        r_low=[]
        g1_low=[]
        g2_low=[]
        b_low=[]

        gt=[]
        
        for_amplifier=[]

        for gener in range(1):
            
            if random.randint(0, 100)>50:
                flip_flag = False
            else:
                flip_flag = False

            if random.randint(0, 100)<20:
                v_flag = False
            else:
                v_flag = False

        #     print(H)
        
#             H,W = img.shape

            i = 0#random.randint(0, (H-self.opt['patch']-2)//2)*2
            j = 0#random.randint(0,(W-self.opt['patch']-2)//2)*2

            img_low = img_loww[i:i+H,j:j+W]
            img_gt = img_gtt[i:i+H,j:j+W,:]
            
            if flip_flag:
                img_gt = np.flip(img_gt, 0).copy()
                img_low = np.flip(img_low, 0).copy()

            if v_flag:
                img_gt = np.flip(img_gt, 1).copy()
                img_low = np.flip(img_low, 1).copy()

            img_low_r = img_low[0:H:2,0:W:2]
            img_low_g1 = img_low[0:H:2,1:W:2]
            img_low_g2 = img_low[1:H:2,0:W:2]
            img_low_b = img_low[1:H:2,1:W:2]
            
            for_amplifier.append(torch.from_numpy(np.transpose(np.dstack((img_low_r,img_low_g1,img_low_g2,img_low_b)), [2, 0, 1])).float())
            

            img_gt_avg = np.zeros((H//8,W//8,int(64*3))).astype(np.float32)

            r_avg = np.zeros((H//16,W//16,64)).astype(np.float32)
            g1_avg = np.zeros((H//16,W//16,64)).astype(np.float32)
            g2_avg = np.zeros((H//16,W//16,64)).astype(np.float32)
            b_avg = np.zeros((H//16,W//16,64)).astype(np.float32)

            count_gt=0
            count_raw = 0
            for ii in range(8):
                for jj in range(8):

                    img_gt_avg[:,:,count_gt:count_gt+3] = img_gt[ii:H:8,jj:W:8,:]
                    count_gt=count_gt+3
        #             print(count_gt)

                    r_avg[:,:,count_raw] = img_low_r[ii:H//2:8,jj:W//2:8]
                    g1_avg[:,:,count_raw] = img_low_g1[ii:H//2:8,jj:W//2:8]
                    g2_avg[:,:,count_raw] = img_low_g2[ii:H//2:8,jj:W//2:8]
                    b_avg[:,:,count_raw] = img_low_b[ii:H//2:8,jj:W//2:8]
                    count_raw=count_raw+1
        #             print('{},{},{}'.format(count_raw,ii,jj))

            
            gt.append(torch.from_numpy((np.transpose(img_gt_avg, [2, 0, 1]))).float())
            r_low.append(torch.from_numpy((np.transpose(r_avg, [2, 0, 1]))).float())
            g1_low.append(torch.from_numpy((np.transpose(g1_avg, [2, 0, 1]))).float())
            g2_low.append(torch.from_numpy((np.transpose(g2_avg, [2, 0, 1]))).float())
            b_low.append(torch.from_numpy((np.transpose(b_avg, [2, 0, 1]))).float())
            
        
        return gt, r_low, g1_low, g2_low, b_low, for_amplifier

In [5]:
obj_train = get_data(opt)
dataloader_train = DataLoader(obj_train, batch_size=opt['batch_size'], shuffle=opt['Shuffle'], num_workers=opt['workers'], pin_memory=opt['Pin_memory'])



In [6]:
class ResBlock(nn.Module):
    
    def __init__(self,in_c):
        super(ResBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_c,in_c, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_c,in_c, kernel_size=3, stride=1, padding=1, bias=True)

    def forward(self, x):
        
        identity = x

        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        out = out + identity
        
        return out
    
class make_dense(nn.Module):
    
    def __init__(self, nChannels=64, growthRate=32, kernel_size=3):
        super(make_dense, self).__init__()
        self.conv = nn.Conv2d(nChannels, growthRate, kernel_size=kernel_size, padding=(kernel_size-1)//2, bias=False)
    def forward(self, x):
        out = F.relu(self.conv(x))
        out = torch.cat((x, out), 1)
        return out
    
# Residual dense block (RDB) architecture
class RDB(nn.Module):
    def __init__(self, nChannels=64, nDenselayer=6, growthRate=32):
        super(RDB, self).__init__()
        nChannels_ = nChannels
        modules = []
        for i in range(nDenselayer):    
            modules.append(make_dense(nChannels_, growthRate))
            nChannels_ += growthRate 
        self.dense_layers = nn.Sequential(*modules)    
        self.conv_1x1 = nn.Conv2d(nChannels_, nChannels, kernel_size=1, padding=0, bias=False)
    def forward(self, x):
        out = self.dense_layers(x)
        out = self.conv_1x1(out)
        out = out + x
        return out
    

    
class amplifier(nn.Module):
# We take this opportunity to show that our Network is insensitive to amplifier choice. For example in 'train.py' we use the one kind of amplifier. Here however we use another variant of that and find no change in the performance.     
    def __init__(self,channels):
        super(amplifier, self).__init__()
        
        self.relu = nn.Threshold(threshold=0, value=0.0001, inplace=True)
        
        # size:256
        self.conv_pre = nn.Sequential(
            nn.Conv2d(channels,16, kernel_size=3, stride=1, padding=1, bias=True),
            # size:128
#             nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1, bias=True)
#             # size:64
#             nn.Conv2d(32,64, kernel_size=3, stride=2, padding=1, bias=True),
#             # size:32
        )
        
        self.conv_post = nn.Sequential(
            nn.Conv2d(16,128, kernel_size=1, stride=1, padding=0, bias=True),
            nn.Conv2d(128,1, kernel_size=1, stride=1, padding=0, bias=True)
        )
        
        
    def forward(self, x):
        
#         print(x.size())
#         identity = x
        
        x = self.conv_pre(x)
        
        # (global average pooling)
        x = F.avg_pool2d(x, x.size()[2:])
        
        # interact
        gamma = self.relu(self.conv_post(x))
#         print(gamma)
        
#         x = identity*gamma
#         print(x.size())
        return gamma
    



In [7]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.amplifier = amplifier(4)
        
        self.relu = nn.ReLU(inplace=True)
        
        self.RDBr = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
        self.RDBg1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
        self.RDBg2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
        self.RDBb = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
        
        self.before_identity = nn.Conv2d(in_channels=int(4*64), out_channels=64, kernel_size=1, stride=1, bias=False)
        self.after_rdb = nn.Conv2d(in_channels=int(3*64), out_channels=64, kernel_size=1, stride=1, bias=False)
        
        self.RDB1 = RDB(nChannels=64, nDenselayer=6, growthRate=32)
        self.RDB2 = RDB(nChannels=64, nDenselayer=6, growthRate=32)
        self.RDB3 = RDB(nChannels=64, nDenselayer=6, growthRate=32)
        
        self.final = nn.Sequential(
            nn.PixelShuffle(2),
            RDB(nChannels=16, nDenselayer=6, growthRate=32),
            nn.Conv2d(in_channels=16, out_channels=int(64*3), kernel_size=3, stride=1, padding=1, bias=True),
#             self.relu
        ) 
        
        
        
    def forward(self,r_low,g1_low,g2_low,b_low,for_amplifier):
        
#         gamma = self.amplifier(torch.cat((r_low,g1_low,g2_low,b_low),dim=1))

        gamma = self.amplifier(for_amplifier)
        
        r_low = self.relu(self.RDBr(r_low*gamma))
        g1_low = self.relu(self.RDBg1(g1_low*gamma))
        g2_low = self.relu(self.RDBg2(g2_low*gamma))
        b_low = self.relu(self.RDBb(b_low*gamma))
        
        alll=self.before_identity(torch.cat((r_low,g1_low,g2_low,b_low),dim=1))
        
        identity = alll
        
        rdb1 = self.RDB1(alll)
        rdb2 = self.RDB2(rdb1)
        rdb3 = self.RDB3(rdb2)
        
        alll = self.after_rdb(torch.cat((rdb1,rdb2,rdb3),dim=1))+identity
        
        alll = self.final(alll)
        
        
        return alll,gamma
    
            




In [8]:
class common_functions():
    
    def __init__(self, opt):
        
        self.opt = opt
        self.count = 0
        self.relu = nn.ReLU(inplace=True)
                        
        self.device = torch.device("cuda")
        

        model = Net()#.apply(self.weights_init_kaiming)
        print('Trainable parameters : {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
        
        self.model = model.to(self.device)
        print(self.model)
        print(next(self.model.parameters()).is_cuda)  
        
        checkpoint = torch.load(self.opt['dir_root']+'weights/'+self.opt['exp_name']+'_{}'.format(400000))
        self.model.load_state_dict(checkpoint['model'])
  
    
    def optimize_parameters(self,r_low,g1_low,g2_low,b_low,gt,for_amplifier):
        
        
        
#         num = random.randint(0, 191)
#         self.num = (num//3)*3 
        
        r_low=r_low.to(self.device)
        g1_low=g1_low.to(self.device)
        g2_low=g2_low.to(self.device)
        b_low=b_low.to(self.device)
        gt=gt.to(self.device)
        for_amplifier=for_amplifier.to(self.device)
        
        self.model.eval()
        
        with torch.no_grad():
            beg = time.time()
            pred_output, gamma = self.model(r_low,g1_low,g2_low,b_low,for_amplifier)
            end = time.time()
        
        begg = time.time()
        
        plot_out_GT = torch.zeros(1,3,2848,4256, dtype=torch.float).to(self.device)
        plot_out_pred = torch.zeros(1,3,2848,4256, dtype=torch.float).to(self.device)
        counttt=0
        for ii in range(8):
                for jj in range(8):

                    plot_out_GT[:,:,ii:2848:8,jj:4256:8] = gt[:,counttt:counttt+3,:,:]
                    plot_out_pred[:,:,ii:2848:8,jj:4256:8] = pred_output[:,counttt:counttt+3,:,:]
                    
                    counttt=counttt+3
        
        endd = time.time()

        
        self.count +=1
            
            
        if True:
            
            plot_out_pred = (np.clip(plot_out_pred[0].detach().cpu().numpy().transpose(1,2,0),0,1)*255).astype(np.uint8)
            plot_out_GT = (np.clip(plot_out_GT[0].detach().cpu().numpy().transpose(1,2,0),0,1)*255).astype(np.uint8)
#             print(gt.shape)
#             print(np.dtype(pred_output))
            
                    
            
            
            
            print("PSNR: {0:.3f}, SSIM: {1:.3f}, RMSE:{2:.3f}, time:{3:.5f}, time:{4:.5f}".format(PSNR(plot_out_GT,plot_out_pred), SSIM(plot_out_GT,plot_out_pred,multichannel=True),NRMSE(plot_out_GT,plot_out_pred), end-beg,endd-begg))
            
            # Save images
            imageio.imwrite('/media/mohit/data/mohit/chen_dark_cvpr_18_dataset/Sony/results/AVG-on-1-10-sec-raw2rgb-patch512-fullVGG-RELU_remove-Testing/{}_IMG_PRED.png'.format(self.count),plot_out_pred)
            imageio.imwrite('/media/mohit/data/mohit/chen_dark_cvpr_18_dataset/Sony/results/AVG-on-1-10-sec-raw2rgb-patch512-fullVGG-RELU_remove-Testing/{}_IMG_GT.png'.format(self.count),plot_out_GT)
            
                

In [9]:
gan_model = common_functions(opt)

# We used MATLABs PSNR/SSIM to for getting the metrics. Although same as the ones computed by python below, we found MATLAB more "trustworthy"
for iteration, img in enumerate(dataloader_train):
    gt = img[0]
    r_low = img[1]
    g1_low = img[2]
    g2_low = img[3]
    b_low = img[4]
    for_amplifier = img[5]

    for faster in range(1):

        gan_model.optimize_parameters(r_low[faster],g1_low[faster],g2_low[faster],b_low[faster],gt[faster],for_amplifier[faster])

Trainable parameters : 1171985
Net(
  (amplifier): amplifier(
    (relu): Threshold(threshold=0, value=0.0001, inplace=True)
    (conv_pre): Sequential(
      (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv_post): Sequential(
      (0): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (relu): ReLU(inplace=True)
  (RDBr): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RDBg1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RDBg2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RDBb): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (before_identity): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (after_rdb): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (RDB1): RDB(
    (dense_layers): Sequential(
      (0): make_dense(
        (conv): Conv2d(64, 32, kerne